In [1]:
import importlib
import sys

sys.path.append("..")  # Ensure the parent directory is in the path

from dataclasses import dataclass
from data_preprocessing.dataset_dataloader import create_data_loader
from transformers import AutoTokenizer
from tqdm import tqdm

# --- Local Application/Module Imports ---
import data_loader.data_loader
import data_preprocessing.data_preprocessing
import data_preprocessing.llm_augmentation
import models.models
import visualizations.visualizations
import utils.utils

importlib.reload(data_loader.data_loader)
from data_loader.data_loader import *

importlib.reload(data_preprocessing.data_preprocessing)

importlib.reload(data_preprocessing.llm_augmentation)
from data_preprocessing.llm_augmentation import *

importlib.reload(models.models)
from models.models import *

importlib.reload(visualizations.visualizations)
from visualizations.visualizations import *

importlib.reload(utils.utils)
from utils.utils import *

# --- Notebook Configuration ---
%matplotlib inline
%config InlineBackend.figure_format='retina'

# --- Global Settings ---
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
os.environ["TOKENIZERS_PARALLELISM"] = "false"

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tnorlha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/tnorlha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to /Users/tnorlha/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/tnorlha/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tnorlha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/tnorlha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to /Users/tnorlha/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/tnorlha/nltk_data...
[nltk_data]   Package wordnet is alre

In [2]:
device = get_device()
print(f"Using device: {device}")

Using device: mps


In [4]:
@dataclass
class Config:
    batch_size: int
    model: str
    seed: int
    lr: float
    dropout: float
    attention_dropout: float
    device: str
    num_classes: int
    max_len: int

config = Config(
    batch_size = 16,
    model="FacebookAI/roberta-large",
    seed=RANDOM_SEED,
    lr=3e-5,
    dropout=0.1,
    attention_dropout=0.1,
    device=device,
    num_classes=3,
    max_len=64,
)

def load_model(config: Config, model_name, model_path):
    config.model = model_name

    llm_tokenizer = AutoTokenizer.from_pretrained(config.model)
    llm_model = load_blank_model(config)

    # Load saved model state dict
    state_dict = torch.load(model_path, map_location=device)
    llm_model.load_state_dict(state_dict)

    llm_model.to(device)
    llm_model.eval()


    loss_fn = nn.CrossEntropyLoss().to(device)

    return llm_model, llm_tokenizer, loss_fn

In [10]:
submission_df = load_submission_data()

def create_submission_data_loader_with_model(llm_tokenizer, config, submission_df):
    submission_data_loader = create_data_loader(submission_df, llm_tokenizer, config.max_len, config.batch_size)

    return submission_data_loader

In [13]:
model_name = "microsoft/deberta-v3-base"
model_path = "../fine_tuned_models/deberta-v3-base-all.bin"

llm_model, llm_tokenizer, loss_fn = load_model(config, model_name, model_path)

submission_data_loader = create_submission_data_loader_with_model(llm_tokenizer, config, submission_df)

y_review_texts, y_pred_db, y_pred_probs_db, y_test = get_predictions(
    llm_model,
    submission_data_loader,
    device
)

/Users/tnorlha/miniconda3/lib/python3.12/site-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
label_map = {0: "negative", 1: "neutral", 2: "positive"}
generate_submission(y_pred_db, label_map=label_map)

Submission file saved to ../submissions/submission.csv


In [16]:
model_name = "microsoft/deberta-v3-large"
model_path = "../fine_tuned_models/deberta-v3-large-all.bin"

llm_model, llm_tokenizer, loss_fn = load_model(config, model_name, model_path)

submission_data_loader = create_submission_data_loader_with_model(llm_tokenizer, config, submission_df)

y_review_texts, y_pred_dl, y_pred_probs_dl, y_test = get_predictions(
    llm_model,
    submission_data_loader,
    device
)

/Users/tnorlha/miniconda3/lib/python3.12/site-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
label_map = {0: "negative", 1: "neutral", 2: "positive"}
generate_submission(y_pred_dl, label_map=label_map)

Submission file saved to ../submissions/submission.csv
